In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
dataset = np.loadtxt("6.LOG", delimiter=";", skiprows=1).T
gps_speed = dataset[0, np.nonzero(dataset[0])]
acceleration = dataset[1:4]
rotation_velocity = dataset[4:7]
magnetic_field = dataset[7:10]
sensor_names = ["Speed", "Acceleration", "Angular velocity", "Magnetic field"]
for arr, name  in zip([gps_speed, acceleration, rotation_velocity, magnetic_field], sensor_names):
    print(f"\n------ {name} ------")
    mean = np.mean(arr, axis=1).squeeze()
    cov = np.cov(arr).squeeze()
    std = np.std(arr, axis=1).squeeze()
    print(f"Mean -> {mean}+-{std / arr.shape[1]}")
    print(f"Covariance ->\n{cov}")

In [ ]:
units = ["m/s^2", "rad/s", "\mu T"]
symbol = ["a", "\omega", "m"]
axes = ["x", "y", "z"]
for i, d in enumerate(dataset[1:]):
    plt.style.use("plot_style.mplstyle")
    t = np.arange(d.size) / 10
    i_time = t < 400
    i_sensor = i // 3
    i_axis = i % 3
    plt.figure(figsize=(5,4))
    plt.plot(t[i_time], d[i_time], ls="", marker="o", alpha=0.4, label=f"Measurements ${symbol[i_sensor]}_{axes[i_axis]}(t)$", rasterized=True)
    plt.axhline(np.mean(d[i_time]), color="black", ls="--", label=f"Mean $\mu=\langle {symbol[i_sensor]}_{axes[i_axis]}\\rangle_t$")
    plt.xlabel("Time (s)")
    plt.ylabel(f"{sensor_names[1:][i_sensor]} ${symbol[i_sensor]}_{axes[i_axis]}$ (${units[i_sensor]}$)")
    plt.xlim(0, 400)
    plt.legend()
    plt.savefig(f"figures/calbration_timeseries_{sensor_names[1:][i_sensor]}_{axes[i_axis]}.pdf", dpi=200, bbox_inches="tight")

    fig = plt.figure(figsize=(5,5))
    ax = plt.gca()
    (osm, osr), (slope, intercept, r) = stats.probplot(d[i_time], fit=True)
    plt.plot(osm, osr, ls="", marker="o", alpha=0.2, label="Experiment quantiles", rasterized=True)
    plt.plot(osm, slope * osm + intercept, ls="--", color="black", label="Ref. normal $\mathcal{N}(\mu,\sigma^2)$")
    plt.xlabel("Theoretical quantiles")
    plt.ylabel(f"{sensor_names[1:][i_sensor]} ${symbol[i_sensor]}_{axes[i_axis]}$ (${units[i_sensor]}$)")
    plt.legend()
    plt.savefig(f"figures/calbration_normprob_{sensor_names[1:][i_sensor]}_{axes[i_axis]}.pdf", dpi=200, bbox_inches="tight")